In [1]:
# pbp_parse.py
#
# parse JSON, make game structure, convert to CSV.
# JSON 데이터를 읽어와 게임 상황을 구현하고
# gameday line 하나에 맞는 text row를 생성, CSV 파일에 저장한다.

import os
import json
from collections import OrderedDict
import regex
import csv

import pandas as pd
import numpy as np
import sys

# custom library
from utils import print_progress

%load_ext autoreload
%autoreload 2

In [191]:
game_id = '20190331SKWO02019'
pl = '_pitching.csv'
bl = '_batting.csv'
rl = '_relay.csv'

tcol = ['textOrder', 'seqno', 'text', 'type', 'stuff',
        'pitchId', 'speed', 'referee', 'stadium', 'inn',
        'topbot', 'stance']
pcol = ['pitchId', 'speed', 'crossPlateX',
        'vx0', 'vy0', 'vz0', 'x0', 'z0', 'ax', 'ay', 'az',
        'topSz', 'bottomSz']
scol = ['outPlayer', 'inPlayer', 'shiftPlayer']

encoding = 'cp949' if sys.platform == 'win32' else 'utf8'
pdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{pl}', encoding=encoding)
bdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{bl}', encoding=encoding)
rdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{rl}', encoding=encoding)
rdf = rdf.sort_values(['textOrder', 'seqno'])

game_date = game_id[:8]
away = game_id[8:10]
home = game_id[10:12]
stadium = rdf.stadium.drop_duplicates().values[0]
referee = rdf.referee.drop_duplicates().values[0]

In [192]:
lineups = [[], []] # 초 공격 / 말 공격
fields = [{}, {}] # 초 수비 / 말 수비

bats = [bdf.loc[bdf.homeaway == 'a'], bdf.loc[bdf.homeaway == 'h']]

pits = [pdf.loc[pdf.homeaway == 'a'], pdf.loc[pdf.homeaway == 'h']]

abat_seqno_min = bats[0].groupby('batOrder').seqno.min().tolist()
hbat_seqno_min = bats[1].groupby('batOrder').seqno.min().tolist()

for i in range(9):
    aCode = bats[0].loc[(bats[0].batOrder == i+1) & (bats[0].seqno == abat_seqno_min[i])].pCode.values[0]
    aName = bats[0].loc[(bats[0].batOrder == i+1) & (bats[0].seqno == abat_seqno_min[i])].name.values[0]
    hCode = bats[1].loc[(bats[1].batOrder == i+1) & (bats[1].seqno == hbat_seqno_min[i])].pCode.values[0]
    hName = bats[1].loc[(bats[1].batOrder == i+1) & (bats[1].seqno == hbat_seqno_min[i])].name.values[0]
    aPlayer = {'name': aName, 'code': aCode}
    hPlayer = {'name': hName, 'code': hCode}
    
    aPos = bats[0].loc[bats[0].pCode == aCode].posName.values[0]
    hPos = bats[1].loc[bats[1].pCode == hCode].posName.values[0]
    fields[1][aPos] = aPlayer
    fields[0][hPos] = hPlayer
    
    aLineup = {'name': aName, 'code': aCode, 'pos': aPos}
    hLineup = {'name': hName, 'code': hCode, 'pos': hPos}
    lineups[0].append(aLineup)
    lineups[1].append(hLineup)

aPitcher = {'name': pits[0].iloc[0]['name'], 'code': pits[0].iloc[0].pCode}
hPitcher = {'name': pits[1].iloc[0]['name'], 'code': pits[1].iloc[0].pCode}
fields[1]['투수'] = aPitcher
fields[0]['투수'] = hPitcher

In [193]:
inn = 0
topbot = 'b'

inns = []
topbots = []

for ind in rdf.index:
    row = rdf.loc[ind]
    if row.type == 0:
        inns.append(inn+1) if topbot == 'b' else inns.append(inn)
        topbots.append('t') if topbot == 'b' else topbots.append('b')
        inn = inn+1 if topbot == 'b' else inn
        topbot = 't' if topbot == 'b' else 'b'
    else:
        inns.append(inn)
        topbots.append(topbot)

rdf['inn'] = inns
rdf['topbot'] = topbots

maxSeqnoInPA = rdf.groupby(['inn', 'topbot', 'textOrder']).seqno.transform('max')
rdf['maxSeqnoInPA'] = maxSeqnoInPA

### TODO: 선수 교체

2인 이상 동시 포지션 변경 혹은 교체시 라인업/수비오더 처리 문제.

순서대로 처리하는 과정에서 out/in 코드 처리가 꼬일 가능성 있음.

ex)
1루수 A : 2루수 D(으)로 교체
2루수 B : 3루수 E(으)로 교체
3루수 C : 1루수 F(으)로 교체

수비 포지션뿐 아니라 타선까지 교체하는데 이 과정에서 타순은 다음 순서로 바뀐다.

1. '1루수' A의 타순에 D를 넣는다.
2. 2루수 자리에 D를 넣는다.
3. '2루수' D의 타순을 가져온다.
4. 원래 B 타순에 E를 넣어야 하지만 A 타순에 들어간다.
5. ...

이렇게 해서 망한다.

### 해결책 : 이번에도 스택 쌓아서 before/after 구분한다.

위 ex를 가져오면

before | after
- | -
1루수 A | 2루수 D
2루수 B | 3루수 E
3루수 C | 1루수 F

스택 쌓아서 순서대로 읽고 기록하면 이렇게 된다.

before POS | before ORDER | after POS | after ORDER
- | - | - | -
1루수 A | 1번 A | 2루수 D | 1번 D
2루수 B | 2번 B | 3루수 E | 2번 E
3루수 C | 3번 C | 1루수 F | 3번 F

이러면 after 값만 보고 쓱쓱 바꾸면 된다.

In [194]:
bases = [None, None, None]
afterBases = [None, None, None]
baseChange = [False, False, False]
score = [0, 0] # 초공(어웨이), 말공(홈)
balls, outs, strikes = 0, 0, 0
batterName, batterCode, _pos = lineups[0][0].values()
pitcherName, pitcherCode = fields[1].get('투수').values()
isPitcherBatting = [False, False]

inns = range(1, rdf.inn.max()+1)
topbot = 1 # 0 : 초, 1 : 말
# fields[1] : 말 수비, fields[0] : 초 수비
# lineups[1] : 말 공격 타순, lineups[0] : 초 공격 타순
cur_order = 1

for inn in inns:
    for tb in ['t', 'b']:
        Inn = rdf.loc[(rdf.inn == inn) & (rdf.topbot == tb)]
        if Inn.size == 0:
            break
        bases = [None, None, None]
        afterBases = [None, None, None]
        baseChange = [False, False, False]
        balls, outs, strikes = 0, 0, 0
        
        tos = Inn.textOrder.drop_duplicates().tolist()

        for to in tos:
            PA = rdf.loc[rdf.textOrder == to]
            print(f'TO {to}')
            lenPA = PA.shape[0]

            ind = 0

            while ind < lenPA:
                row = PA.iloc[ind]
                PAEndFlag = False

                ##############################
                ###### type에 따라 파싱 ######
                ##############################
                
                if (row.type == 1):
                    res = parsePitchResult(row.text)
                    if res == 'b':
                        balls += 1
                    elif res == 's':
                        strikes += 1
                    elif res == 'm':
                        strikes += 1
                    elif res == 'f':
                        strikes = strikes+1 if strikes < 2 else 2
                    print(f'\t{row.text}')
                    ind = ind + 1
                elif (row.type == 8):
                    # 타석 시작(x번타자 / 대타)
                    order = row.text.split(' ')[0]
                    batter = row.text.split(' ')[1]
                    if len(order) > 2:
                        cur_order = int(order[0])
                        batterName, batterCode, _pos = lineups[topbot][cur_order - 1].values()
                    else:
                        batterName, batterCode, _pos = lineups[topbot][cur_order - 1].values()
                    print(f'\t{row.text}')
                    ind = ind + 1
                elif ((row.type == 13) | (row.type == 23)):
                    # 타자주자(비득점/득점)
                    PAEndFlag = True
                    runner_stack = []
                    cur_ind = ind
                    cur_row = PA.iloc[cur_ind]
                    while ((cur_row.type == 13) | (cur_row.type == 23) | (cur_row.type == 14) | (cur_row.type == 24)):
                        runner_stack.append(cur_row)
                        cur_ind = cur_ind + 1
                        if cur_ind >= lenPA:
                            break
                        cur_row = PA.iloc[cur_ind]
                    ind = cur_ind
                    for i in range(len(runner_stack)):
                        rrow = runner_stack[i]
                        if (rrow.type == 13) | (rrow.type == 23):
                            res = parseBatterAsRunner(rrow.text) # runner, result, beforebase, afterBase
                            result = parseBatterResult(rrow.text)
                            if res[1] == 'h':
                                score[topbot] += 1
                            elif res[1] == 'o':
                                outs += 1
                            else:
                                afterBases[res[3]-1] = batterCode
                                baseChange[res[3]-1] = True
                            print(f'\t\t{rrow.text} || {result} || {res}')
                        else:
                            res = parseRunnerResult(rrow.text)
                            runnerCode = bases[res[2]-1] if baseChange[res[2]-1] is False else afterBases[res[2]-1]
                            if ((baseChange[res[2]-1] is True) & (afterBases[res[2]-1] == batterCode)) &\
                               ((bases[res[2]-1] is not None) & (bases[res[2]-1] != batterCode)):
                                runnerCode = bases[res[2]-1]
                            
                            if res[1] == 'h':
                                score[topbot] += 1
                            elif res[1] == 'o':
                                outs += 1
                            else:
                                afterBases[res[3]-1] = runnerCode
                                baseChange[res[3]-1] = True
                            
                            if baseChange[res[2]-1] is False:
                                afterBases[res[2]-1] = None
                                baseChange[res[2]-1] = True
                            else:
                                if afterBases[res[2]-1] == runnerCode:
                                    afterBases[res[2]-1] = None

                            print(f'\t\t{rrow.text} || {res}')
                    bases = afterBases
                    afterBases = [None, None, None]
                    baseChange = [False, False, False]
                elif ((row.type == 14) | (row.type == 24)):
                    # 주자(비득점/득점)
                    runner_stack = []
                    cur_runner = None
                    cur_ind = ind
                    cur_row = PA.iloc[cur_ind]
                    while ((cur_row.type == 14) | (cur_row.type == 24)):
                        runner_stack.append(cur_row)
                        cur_ind = cur_ind + 1
                        if cur_ind >= lenPA:
                            break
                        cur_row = PA.iloc[cur_ind]
                    ind = cur_ind
                    for i in range(len(runner_stack)):
                        rrow = runner_stack[i]
                        res = parseRunnerResult(rrow.text)
                        runnerCode = bases[res[2]-1] if baseChange[res[2]-1] is False else afterBases[res[2]-1]
                        if ((baseChange[res[2]-1] is True) & (afterBases[res[2]-1] == batterCode)) &\
                           ((bases[res[2]-1] is not None) & (bases[res[2]-1] != batterCode)):
                            runnerCode = bases[res[2]-1]

                        if res[1] == 'h':
                            score[topbot] += 1
                        elif res[0] == 'o':
                            outs += 1
                        else:
                            afterBases[res[3]-1] = runnerCode
                            baseChange[res[3]-1] = True

                        if baseChange[res[2]-1] is False:
                            afterBases[res[2]-1] = None
                            baseChange[res[2]-1] = True
                        else:
                            if afterBases[res[2]-1] == runnerCode:
                                afterBases[res[2]-1] = None
                        print(f'\t\t=={rrow.text} || {res}')
                    bases = afterBases
                    afterBases = [None, None, None]
                    baseChange = [False, False, False]
                elif (row.type == 0):
                    # 이닝 시작
                    print(f'\t{row.text}')
                    ind = ind + 1
                    topbot = (1 - topbot)
                    pitcherName, pitcherCode = fields[topbot].get('투수').values()
                elif (row.type == 2):
                     # 교체/변경
                    outPlayer, inPlayer, shiftPlayer = None, None, None
                    outPos = row.text.split(' ')[0]
                    inPos = None
                    outCode, inCode, shiftCode = None, None, None

                    if row.text.find('변경') > 0:
                        homeaway = 't' if topbot == 0 else 'b'
                        shiftPlayer = row.text.split(' ')[1].strip()
                        inPos = row.text.split('(')[0].strip().split(' ')[-1]
                        order = None
                        
                        # 바뀌는 선수 code 찾기
                        if outPos == '투수':
                            shiftCode = fields[topbot].get(inPos).get('code')
                        else:
                            for i in range(9):
                                if (lineups[1 - topbot][i].get('name') == shiftPlayer) &\
                                   (lineups[1 - topbot][i].get('pos') == outPos):
                                    shiftCode = lineups[1 - topbot][i].get('code')
                                    order = i
                                    break
                        if inPos == '투수':
                            pitcherCode = shiftCode
                            pitcherName = shiftPlayer
                            fields[topbot]['투수']['code'] = pitcherCode
                            fields[topbot]['투수']['name'] = pitcherName
                            lineups[1-topbot][order]['pos'] = '투수'
                        else:
                            fields[topbot][inPos]['code'] = shiftCode
                            fields[topbot][inPos]['name'] = shiftPlayer
                            lineups[1-topbot][order]['pos'] = inPos
                            
                    else:
                        # 교체
                        # 포지션, 이름 파싱
                        inPos = row.text.split('(')[0].strip().split(' ')[3]
                        inPlayer = row.text.split('(')[0].strip().split(' ')[-1]
                        outPlayer = row.text.split(' ')[1].strip()
                        
                        # 나가는 선수 code 찾기
                        # 대주자, 대타 빼는 경우 order 찾는데 필수
                        if outPos == '투수':
                            outCode = pitcherCode
                        elif inPos == '대타':
                            outCode = lineups[topbot][cur_order - 1].get('code')
                        elif inPos == '대주자':
                            outBase = int(outPos[0])
                            outCode = bases[outBase - 1]
                            # 여기서부터는 야수 교체(수비 교체)
                        elif (outPos == '대주자') or (outPos == '대타'):
                            homeaway = 'h' if topbot == 0 else 'a'
                            outRow = bdf.loc[(bdf.homeaway == homeaway) &
                                             (bdf.posName == outPos) &
                                             (bdf.name == outPlayer)]
                            if inPos == '대타':
                                # 유일한 예외 - 대타를 대타로 바꾸기
                                homeaway = 'a' if topbot == 0 else 'h'                 
                                outRow = bdf.loc[(bdf.homeaway == homeaway) &
                                                 (bdf.name == outPlayer) &
                                                 (bdf.pCode == batterCode)]
                            if len(outRow) == 1:
                                outCode = int(outRow.pCode)
                            elif len(outRow) > 1:
                                for i in range(len(outRow)):
                                    oRow = outRow.loc[i]
                                    order = int(oRow.batOrder)
                                    tempRow = bdf.loc[(bdf.batOrder == order) & (bdf.homeaway == homeaway)]
                                    if len(tempRow.loc[tempRow.name == inPlayer]) < 1:
                                        continue
                                    else:
                                        outCode = int(oRow.pCode)
                                        break
                        else:
                            outCode = fields[topbot][outPos].get('code')
                        
                        # 들어가는 선수 code 찾기
                        if inPos == '대주자':
                            homeaway = 'a' if topbot == 0 else 'h'
                            if outCode is not None:
                                order = int(bdf.loc[bdf.pCode == outCode].batOrder)
                                seqno = int(bdf.loc[bdf.pCode == outCode].seqno)
                                inCode = int(bdf.loc[(bdf.homeaway == homeaway) &
                                                     (bdf.batOrder == order) &
                                                     (bdf.seqno > seqno)].head(1).pCode)
                            else:
                                order = int(bdf.loc[(bdf.name == outPlayer) & (bdf.homeaway == homeaway)].batOrder)
                                seqno = int(bdf.loc[(bdf.name == outPlayer) & (bdf.homeaway == homeaway)].seqno)
                                inCode = int(bdf.loc[(bdf.homeaway == homeaway) &
                                                     (bdf.batOrder == order) &
                                                     (bdf.seqno > seqno)].head(1).pCode)
                        elif inPos == '대타':
                            homeaway = 'a' if topbot == 0 else 'h'
                            order = int(outPos[0])
                            seqno = int(bdf.loc[bdf.pCode == outCode].seqno)
                            inCode = int(bdf.loc[(bdf.homeaway == homeaway) &
                                                 (bdf.batOrder == order) &
                                                 (bdf.seqno > seqno)].head(1).pCode)
                        elif inPos == '투수':
                            homeaway = 'h' if topbot == 0 else 'a'
                            outCode = fields[topbot].get('투수').get('code')
                            seqno = int(pdf.loc[pdf.pCode == outCode].seqno)
                            inCode = int(pdf.loc[(pdf.homeaway == homeaway) & (pdf.seqno > seqno)].head(1).pCode)
                        else:
                            homeaway = 'h' if topbot == 0 else 'a'
                            order = int(bdf.loc[bdf.pCode == outCode].batOrder)
                            seqno = int(bdf.loc[bdf.pCode == outCode].seqno)
                            inCode = int(bdf.loc[(bdf.homeaway == homeaway) &
                                                 (bdf.batOrder == order) &
                                                 (bdf.seqno > seqno)].head(1).pCode)

                        # lineup / fields 교체
                        if inPos == '투수':
                            pitcherCode = inCode
                            pitcherName = inPlayer
                            fields[topbot]['투수']['code'] = pitcherCode
                            fields[topbot]['투수']['name'] = pitcherName
                            
                            if isPitcherBatting[topbot]:
                                for i in range(9):
                                    if (lineups[1-topbot][i].get('pos') == outPos) &\
                                       (lineups[1-topbot][i].get('code') == outCode):
                                        lineups[1-topbot][i]['code'] = pitcherCode
                                        lineups[1-topbot][i]['name'] = pitcherName
                                        lineups[1-topbot][i]['pos'] = '투수'
                                        break
                            elif outPos != '투수':
                                for i in range(9):
                                    if (lineups[1-topbot][i].get('pos') == outPos) &\
                                       (lineups[1-topbot][i].get('code') == outCode):
                                        lineups[1-topbot][i]['code'] = pitcherCode
                                        lineups[1-topbot][i]['name'] = pitcherName
                                        lineups[1-topbot][i]['pos'] = '투수'
                                        isPitcherBatting[topbot] = True
                                        break
                        elif inPos == '대타':
                            batterCode = inCode
                            batterName = inPlayer
                            order = int(outPos[0])
                            lineups[topbot][cur_order - 1]['code'] = batterCode
                            lineups[topbot][cur_order - 1]['name'] = batterName
                            lineups[topbot][cur_order - 1]['pos'] = inPos
                        elif inPos == '대주자':
                            outBase = int(outPos[0])
                            bases[outBase - 1] = inCode
                            
                            for i in range(9):
                                if lineups[topbot][i].get('code') == outCode:
                                    lineups[topbot][i]['code'] = inCode
                                    lineups[topbot][i]['name'] = inPlayer
                                    lineups[topbot][i]['pos'] = inPos
                                    break
                        else:
                            fields[topbot][inPos]['code'] = inCode
                            fields[topbot][inPos]['name'] = inPlayer
                            
                            for i in range(9):
                                if lineups[1 - topbot][i].get('code') == outCode:
                                    lineups[1 - topbot][i]['code'] = inCode
                                    lineups[1 - topbot][i]['name'] = inPlayer
                                    lineups[1 - topbot][i]['pos'] = inPos
                                    break
                    print(f'\t{row.text}')
                    ind = ind + 1
                elif (row.type == 7):
                    print(f'\t{row.text}')
                    ind = ind + 1
                else:
                    ind = ind + 1


TO 0
	1회초 SK 공격
TO 1
	1번타자 노수광
	1구 스트라이크
	2구 스트라이크
	3구 파울
	4구 헛스윙
		노수광 :  삼진 아웃  || ('삼진', '삼진') || ['노수광', 'o', 0, None]
TO 2
	2번타자 한동민
	1구 스트라이크
	2구 볼
	3구 파울
	4구 파울
	5구 헛스윙
		한동민 :  삼진 아웃  || ('삼진', '삼진') || ['한동민', 'o', 0, None]
TO 3
	3번타자 김강민
	1구 스트라이크
	2구 파울
	3구 타격
		김강민 : 3루수 땅볼 아웃 (3루수->1루수 송구아웃) || ('포스 아웃', '포스 아웃') || ['김강민', 'o', 0, None]
TO 4
	1회말 키움 공격
TO 5
	1번타자 이정후
	1구 스트라이크
	2구 볼
	3구 파울
	4구 파울
	5구 볼
	6구 볼
	7구 볼
		이정후 :  볼넷 || ('볼넷', '볼넷') || ['이정후', 'a', 0, 1]
TO 6
	2번타자 서건창
	1구 볼
	2구 스트라이크
	3구 스트라이크
	4구 파울
	5구 파울
	6구 볼
	7구 파울
	8구 파울
	9구 파울
	10구 헛스윙
		서건창 :  삼진 아웃  || ('삼진', '삼진') || ['서건창', 'o', 0, None]
TO 7
	3번타자 김하성
	1구 볼
	2구 볼
	3구 볼
	4구 볼
		김하성 :  볼넷 || ('볼넷', '볼넷') || ['김하성', 'a', 0, 1]
		1루주자 이정후 : 2루까지 진루 || ['이정후', 'a', 1, 2]
TO 8
	4번타자 박병호
	1구 볼
	2구 스트라이크
	3구 파울
	4구 볼
	5구 볼
	6구 타격
		박병호 : 우익수 플라이 아웃  || ('필드 아웃', '필드 아웃') || ['박병호', 'o', 0, None]
		2루주자 이정후 : 3루까지 진루 || ['이정후', 'a', 2, 3]
TO 9
	5번타자 샌즈
	1구 스트라이크
	2구 스트라이크
	3구 볼
	4구 헛스윙
		샌즈 :  삼진 아웃  || ('삼진'

		노수광 : 투수 땅볼 아웃 (투수->1루수 송구아웃) || ('포스 아웃', '포스 아웃') || ['노수광', 'o', 0, None]
TO 91
	2번타자 한동민
	1구 볼
	2구 파울
	3구 파울
	4구 볼
	5구 볼
	6구 파울
	7구 타격
		한동민 : 유격수 플라이 아웃  || ('필드 아웃', '필드 아웃') || ['한동민', 'o', 0, None]
TO 92
	8회말 키움 공격
TO 93
	1번타자 이정후
64805 이승진 투수 -> 79130 강지광 투수
	투수 이승진 : 투수 강지광 (으)로 교체
	1구 볼
	2구 스트라이크
	3구 볼
	4구 타격
		이정후 : 좌익수 파울플라이 아웃  || ('필드 아웃', '필드 아웃') || ['이정후', 'o', 0, None]
TO 94
	2번타자 서건창
	1구 스트라이크
	2구 타격
		서건창 : 중견수 앞 1루타 || ('안타', '안타') || ['서건창', 'a', 0, 1]
TO 95
	3번타자 김하성
	1구 타격
		김하성 : 3루수 앞 땅볼로 출루  || ('포스 아웃', '포스 아웃') || ['김하성', 'a', 0, 1]
		1루주자 서건창 :  포스아웃 (3루수->2루수 2루 터치아웃) || ['서건창', 'o', 1, None]
TO 96
	4번타자 박병호
	1구 헛스윙
	2구 타격
		박병호 : 유격수 앞 땅볼로 출루  || ('포스 아웃', '포스 아웃') || ['박병호', 'a', 0, 1]
		1루주자 김하성 :  포스아웃 (유격수->2루수 2루 터치아웃) || ['김하성', 'o', 1, None]
TO 97
	9회초 SK 공격
TO 98
	3번타자 김강민
76430 김상수 투수 -> 62360 김동준 투수
	투수 김상수 : 투수 김동준 (으)로 교체
	1구 타격
		김강민 : 유격수 뒤 1루타 || ('안타', '안타') || ['김강민', 'a', 0, 1]
TO 99
	4번타자 이재원
	1구 스트라이크
	2구 헛스윙
	3구 스트라이크
		이재원 :  삼진 

# Structure

Top-Bottom 식으로 밑그림을 그리고 가보자.

## Read Files

경기 별로 파일은 3가지가 저장된다.
1. pbp_data/{Year}/{Month}/{gameID}_relay.csv
2. pbp_data/{Year}/{Month}/{gameID}_batting.csv
3. pbp_data/{Year}/{Month}/{gameID}_pitching.csv



# 1. `relay` 중계 파일 구조

`csv` 형식으로 저장했는데 각 컬럼의 의미는 이렇다.

columns| definition
-|-
textOrder| 텍스트 순번(타석, 메시지 단위로 구분)
seqno| 같은 `textOrder` 안에서 출력 순서
text | 중계 텍스트 내용
type | 텍스트 타입
stuff | 구종
pitchNum | 타석 투구수(신뢰 x)
pitchResult | 스트(T)/볼(B)/파울(F)/헛스윙(S)
pitchId | 트래킹데이터 고유값 ID
speed | 문자중계 찍히는 구속
referee | 구심
stadium | 구장
outPlayer | 교체시/나가는 선수
inPlayer | 교체시/들어온 선수
shiftPlayer | 포변/포변 선수
inn | 이닝(신뢰 x)
ballcount | 타석 투구수(신뢰 x)
기타 | vx0, vy0, ... 트래킹 데이터

## type 의미

- `0` : x회초(말) xx 공격
- `1` : 투구 (x구 ...)
- `2` : 교체
- `7` : 시스템 메시지 (비디오 판독, ...)
- `8` : x번타자 xxx
- `13` : 타자주자 타석 결과(득점 없는 타석)
- `14` : 주자 주루/아웃(득점 없는 타석)
- `23` : 타자주자 타석 결과(득점 있는 타석)
- `24` : 주자 주루/아웃(득점 있는 타석)
- `44` : 파울 에러 (pass)
- `99` : 경기 종료 메시지

# 2. `batting` 파일 구조 / 야수 관련 `object` 설정

`csv` 형식이며 각 컬럼 의미는 이렇다.

columns | definition
-|-
name | 선수 이름
pCode | 선수 코드
pos | 포지션(한글) - 선발선수는 선발시 포지션, 나머지는 경기종료시 포지션
hitType | 투타
seqno | 해당 타순 선수들의 출전 순서(낮으면 선발)
batOrder | 타순
homeaway | `h`는 홈, `a`는 어웨이
ab, hit, ... | 경기 기록

## 초기 `object` 설정

`csv` 파일을 `pd.DataFrame` 형식으로 로드해둔다.

로드 이후 다음 순서로 경기 시작시 배팅 오더/수비 라인업 오브젝트를 구성한다.

1. `batOrder` 순으로 순회한다.
2. 각 `batOrder`에서 `seqno` 값이 가장 낮은 row를 뽑아 라인업 오더에 넣는다.
    - 2-1. `name`, `pCode`만 넣어둔다. 세부 정보 필요시 `pCode`로 `DataFrame`을 조회한다.
    - 2-2. 해당 row의 포지션으로 선발 수비 라인업도 구성한다.
3. 수비라인업의 투수 설정을 한다. (#3. `pitching` 파일 구조 참조)

## 조회하는 경우
경기 중에 배팅 오더/수비 라인업 오브젝트를 조회하는 경우는 다음이 있다.

1. 선수 교체시
    - 1-1. 대타 : 배팅 오더 교체, 나간선수 수비 라인업 `nullify`
    - 1-2. 투교 : 수비 라인업 `refresh`
    - 1-3. 포변 : 수비 라인업 `refresh`
2. 1구 단위 row 기록할 때 : 수비 라인업 조회하고 기록
3. 타석 바뀔 때: 타순 보고 선수 대조

# 3. `pitching ` 파일 구조 / 투수 관련 `object` 설정

`csv` 형식이며 각 컬럼 의미는 이렇다.

columns | definition
-|-
name | 선수 이름
pCode | 선수 코드
hitType | 투타
seqno | 출전 순서(낮으면 선발)
homeaway | `h`는 홈, `a`는 어웨이
inn, run, er, ... | 경기 기록


## 초기 `object` 설정

`csv` 파일을 `pd.DataFrmae` 형식으로 로드해둔다.

로드 이후 다음 순서로 경기 시작시 배팅 오더/수비 라인업 오브젝트를 구성한다.

1. 야수쪽 수비 라인업 설정을 마친뒤 투수도 수비 라인업에 설정한다.
  1-1. 야수와 마찬가지로 `name`, `pCode`만 넣어둔다.
  1-2. `seqno` 가장 낮은 투수를 집어넣는다.

## 조회하는 경우

경기 중에 투수 라인업을 조회하는 경우는 다음이 있다.

1. 선수 교체시
  1-1. 대타: 아주 가끔 투수가 대타로 나갈 때 있다.
  1-2. 투교: 수비 라인업 `refresh`
2. 1구 단위 row 기록할 때 : 수비 라인업 조회하고 기록

# 4. 로드 이후 순서

1. `textOrder`로 중계 텍스트를 나눈다.
2. `seqno` 순서로 텍스트를 로드, 파싱한다.
3. 파싱 결과에 따라 game status를 변경
4. 텍스트 전체 파싱, 진행 끝나면 csv로 저장

# 파싱 순서

1. 타석 단위로 스택 처리, `type` 따라 달라짐
    - `type`에 따라서...
        - `0` : 이닝 status refresh
        - `1` : 투구 결과 및 데이터 기록
        - `2` : 교체 반영 -> 배팅 오더, 수비 라인업
        - `7` : 시스템 메시지
            - continue
        - `8` : x번타자/대타 xxx -> game status에서 현재 타자 이름 변경
        - `13` : __타자주자__ 타석 결과(득점 없는 타석)
            - 타석 stack 생성
            - 타석 결과 나왔다는 flag True로 바꾸기
            - base 결과, out 결과, score 결과 stack에 쌓는다
            - **곧바로 출력하지 않는다**: PA 끝까지 stack에 쌓아서 주루 결과까치 처리 후 출력
            - 결과도 즉각 game status에 반영하지 않는다            
        - `14` : __주자__ 주루/아웃(득점 없는 타석)
            - 타석 결과 flag 확인 : False인 경우 새로 stack 생성
                - 타석 도중에 나올 수 있기 때문(도루 성공&실패, 견제사, 보크, 폭투 등)
            - stack 다 쌓은 다음에 처리
        - `23` : __타자주자__ 타석 결과(득점 있는 타석)
            - `13`과 동일, '홈인' 메시지는 득점 반영 / 주자 베이스 처리
        - `24` : __주자__ 주루/아웃(득점 있는 타석)
            - `14`와 동일, '홈인' 메시지는 득점 반영 / 주자 베이스 처리
        - `44` : 파울 에러 (pass)
            - continue
        - `99` : 경기 종료 메시지
            - continue
2. 스택 처리
    - print stack을 만들고, 여기에 출력 row를 쌓아둠
    - pitch 들어오면 현재 game status 출력해 row 적립, game status 수정
    - 주자 스택은 모아서 처리
        - 중간에 나오는 주자 스택은 모아서 따로 row로 출력
            - row 하나로 출력할 수 있도록 처리
        - 마지막에 나오는 스택은 타석 결과가 있으면 함께 처리
            - 타석 결과 없을 수도 있음(견제사 3아웃 등)
    - 13, 14, 23, 24 들어오면 tail row의 description 수정(append), game status 수정
    - 13, 14, 23, 24에서 주자 처리는 다음 순서로
        - 13, 14, 23, 24 들어오면 current runner로 설정
        - while current row type in (13, 14, 23, 24)
          - if current runner = None
              - current runner <= row runner
              - current runner의 play 처리, location 변경
          - if row runner != current runner
              - current runner의 description과 after play location을 확정
              - current runner <= row runner
              - current runner의 play 처리, location 변경

# 주루 경우의 수 정리

before | after | 출력
-|-|-
. . . | O . . | 1루타; 볼넷; 실책으로 출루; 몸에 맞는 볼; 낫아웃 폭투; 낫아웃 포일; 내야안타; 번트안타; 타격방해
. . . | . O . | 2루타; 1루타/내야안타/번트안타/낫아웃 폭투/낫아웃 포일 + 실책으로 출루
. . . | . . O | 3루타; 1루타/2루타/내야안타/번트안타/낫아웃 폭투/낫아웃 포일 + 실책으로 출루
. . . | . . . | 

# 타석 결과, 주자 주루 stack 처리

1. 타석 마지막 나오는 `13`, `14`, `23`, `24`
    - 타석 결과 있는 경우 : `13`, `23`
        1. 끝까지 row 스태킹
        2. 타석 결과 row : 결과 획득
        3. 타석 결과 row : 주루 처리
        4. (option) 주루 결과 row : 주루 처리
        5. 반복
    - 타석 결과 없는 경우 : `14`, `24`
        1. 끝까지 row 스태킹
        2. 주루 결과 row : 주루 처리 append append
        3. 반복 후 출력
2. 타석 중간에 나오는 `14`, `24`
    1. while로 투구 전까지 결과 스태킹
    2. 결과 처리

# 타석 결과 종류 정리

keyword | 간단 분류 | 상세 분류
-|-|-
삼진 | 삼진 | 삼진
볼넷 | 볼넷 | 볼넷
자동 고의 4구 | 고의 4구 | 고의 4구
고의4구 | 고의 4구 | 고의 4구
몸에 | 몸에 맞는 공 | 몸에 맞는 공
1루타 | 안타 | 안타
내야안타 | 내야 안타 | 내야 안타
번트안타 | 번트 안타 | 번트 안타
안타 | 안타 | 안타
2루타 | 2루타 | 2루타
3루타 | 3루타 | 3루타
홈런 | 홈런 | 홈런
낫아웃 폭투 | 낫아웃 출루 | 낫아웃 폭투
낫아웃 포일 | 낫아웃 출루 | 낫아웃 포일
낫 아웃 | 삼진 | 낫아웃 삼진
낫아웃 다른주자 수비 실책 | 낫아웃 출루 | 낫아웃 다른 주자 수비 실책
낫아웃 다른주자 수비 | 낫아웃 출루 | 낫아웃 다른 주자 포스 아웃
땅볼로 출루 | 포스 아웃 | 포스 아웃
땅볼 아웃 | 포스 아웃 | 포스 아웃
플라이 아웃 | 필드 아웃 | 필드 아웃
인필드 | 필드 아웃 | 인필드 플라이
파울플라이 | 필드 아웃 | 파울 플라이 아웃
라인드라이브 아웃 | 필드 아웃 | 라인드라이브 아웃
번트 아웃 | 필드 아웃 | 번트 아웃
병살타 | 병살타 | 병살타
희생번트 아웃 | 희생 번트 | 희생 번트
희생플라이 아웃 | 희생 플라이 | 희생 플라이
희생플라이아웃 | 희생 플라이 | 희생 플라이
쓰리번트 | 삼진 | 쓰리번트 삼진
타구맞음 | 필드 아웃 | 타구맞음 아웃
희생번트 실책 | 실책 | 실책
희생번트 야수선택 | 야수 선택 | 야수 선택
야수선택 | 야수 선택 | 야수 선택
실책 | 실책 | 실책
타격방해 | 타격 방해 | 타격 방해
삼중살 | 삼중살 | 삼중살
부정타격 | 필드 아웃 | 부정 타격 아웃
번트 | 번트 안타 | 안타

# 타석 결과 예외
1. 병살타
    - 선행주자 2명 포스아웃 : 병살타가 아닌 '땅볼로 출루'로 기록.
        - [예시](https://sports.news.naver.com/gameCenter/textRelay.nhn?gameId=20090705HTHH0&category=kbo) https://sports.news.naver.com/gameCenter/textRelay.nhn?gameId=20090705HTHH0&category=kbo
        
        

In [31]:
def parsePitchResult(text):
    res = text.split(' ')[-1]
    if res == '볼':
        return 'b'
    elif res == '스트라이크':
        return 's'
    elif res == '헛스윙':
        return 'm'
    elif res == '파울':
        return 'f'
    elif res == '타격':
        return 'h'
    else:
        return None

In [30]:
def parseBatterResult(text):
    for tup in batterResults:
        if text.find(tup[0]) >= 0:
            return tup[1], tup[2]

In [29]:
def parseBatterAsRunner(text):
    runner = text.split(' ')[0]
    
    result = 'o' if text.find('아웃') > 0 else 'a'
    result = 'h' if text.find('홈런') > 0 else result
    
    beforeBase = 0
    afterBase = None
    if any([s in text for s in ['안타', '1루타', '볼넷', '출루', '낫아웃 포일', '낫아웃 폭투', '몸에 맞는', '낫아웃 다른주자']]):
        afterBase = 1
    elif '2루타' in text:
        afterBase = 2
    elif '3루타' in text:
        afterBase = 3
    elif '홈런' in text:
        afterBase = 4
    elif '아웃' in text:
        afterBase = None
    
    return [runner, result, beforeBase, afterBase]

In [28]:
def parseRunnerResult(text):
    runner = text.split(' ')[1]
    
    result = 'o' if text.find('아웃') > 0 else 'a' # o : out, a : advance
    result = 'h' if text.find('홈인') > 0 else result # 'h' : home-in
    
    beforeBase = int(text[0])
    afterBase = None if result != 'a' else int(text[text.find('루까지')-1])
    afterBase = 4 if result == 'h' else afterBase
    
    return [runner, result, beforeBase, afterBase]

In [14]:
batterResults = [
    ['삼진', '삼진', '삼진'],
    ['볼넷', '볼넷', '볼넷'],
    ['자동 고의 4구', '고의 4구', '고의 4구'],
    ['고의4구', '고의 4구', '고의 4구'],
    ['몸에', '몸에 맞는 공', '몸에 맞는 공'],
    ['1루타', '안타', '안타'],
    ['내야안타', '내야 안타', '내야 안타'],
    ['번트안타', '번트 안타', '번트 안타'],
    ['안타', '안타', '안타'],
    ['2루타', '2루타', '2루타'],
    ['3루타', '3루타', '3루타'],
    ['홈런', '홈런', '홈런'],
    ['낫아웃 폭투', '낫아웃 출루', '낫아웃 폭투'],
    ['낫아웃 포일', '낫아웃 출루', '낫아웃 포일'],
    ['낫 아웃', '삼진', '낫아웃 삼진'],
    ['낫아웃 다른주자 수비 실책', '낫아웃 출루', '낫아웃 다른 주자 수비 실책'],
    ['낫아웃 다른주자 수비', '낫아웃 출루', '낫아웃 다른 주자 포스 아웃'],
    ['땅볼로 출루', '포스 아웃', '포스 아웃'],
    ['땅볼 아웃', '포스 아웃', '포스 아웃'],
    ['플라이 아웃', '필드 아웃', '필드 아웃'],
    ['인필드', '필드 아웃', '인필드 플라이'],
    ['파울플라이', '필드 아웃', '파울 플라이 아웃'],
    ['라인드라이브 아웃', '필드 아웃', '라인드라이브 아웃'],
    ['번트 아웃', '필드 아웃', '번트 아웃'],
    ['병살타', '병살타', '병살타'],
    ['희생번트 아웃', '희생 번트', '희생 번트'],
    ['희생플라이 아웃', '희생 플라이', '희생 플라이'],
    ['희생플라이아웃', '희생 플라이', '희생 플라이'],
    ['쓰리번트', '삼진', '쓰리번트 삼진'],
    ['타구맞음', '필드 아웃', '타구맞음 아웃'],
    ['희생번트 실책', '실책', '실책'],
    ['희생번트 야수선택', '야수 선택', '야수 선택'],
    ['야수선택', '야수 선택', '야수 선택'],
    ['실책', '실책', '실책'],
    ['타격방해', '타격 방해', '타격 방해'],
    ['삼중살', '삼중살', '삼중살'],
    ['부정타격', '필드 아웃', '부정 타격 아웃'],
    ['번트', '번트 안타', '안타'],
]

In [15]:
class game_status:
    def __init__ (self):
        self.score_home = 0
        self.score_away = 0
        self.balls = 0
        self.strikes = 0
        self.outs = 0
        self.on_1b = None
        self.on_2b = None
        self.on_3b = None
        self.inn = 0
        self.inn_topbot = 't'
        self.game_date = None
        self.away = None
        self.home = None
        self.stadium = None
        self.referee = None
        self.pa_number = 0
        self.pitch_number = 0
        self.away_field = None
        self.away_order = None
        self.home_field = None
        self.home_order = None
        self.pitchData = None # pointer

    def setting(self, game_id, text_df, away_lineup_df, home_lineup_df):
        self.game_date = game_id[:8]
        self.away = game_id[8:10]
        self.home = game_id[10:12]
        self.stadium = text_df.stadium.drop_duplicates().values[0]
        self.referee = text_df.referee.drop_duplicates().values[0]
        
        away_order = []
        home_order = []
        away_field = {}
        home_field = {}

        abat = bdf.loc[bdf.homeaway == 'a']
        hbat = bdf.loc[bdf.homeaway == 'h']

        apit = pdf.loc[pdf.homeaway == 'a']
        hpit = pdf.loc[pdf.homeaway == 'h']

        abat_seqno_min = abat.groupby('batOrder').seqno.min().tolist()
        hbat_seqno_min = hbat.groupby('batOrder').seqno.min().tolist()

        for i in range(9):
            aCode = abat.loc[(abat.batOrder == i+1) & (abat.seqno == abat_seqno_min[i])].pCode.values[0]
            aName = abat.loc[(abat.batOrder == i+1) & (abat.seqno == abat_seqno_min[i])].name.values[0]
            hCode = hbat.loc[(hbat.batOrder == i+1) & (hbat.seqno == hbat_seqno_min[i])].pCode.values[0]
            hName = hbat.loc[(hbat.batOrder == i+1) & (hbat.seqno == hbat_seqno_min[i])].name.values[0]
            aPlayer = {'name': aName, 'code': aCode}
            hPlayer = {'name': hName, 'code': hCode}
            away_order.append(aPlayer)
            home_order.append(hPlayer)

            aPos = abat.loc[abat.pCode == aCode].posName.values[0]
            hPos = hbat.loc[hbat.pCode == hCode].posName.values[0]
            away_field[aPos] = aPlayer
            home_field[hPos] = hPlayer

        aPitcher = {'name': apit.iloc[0]['name'], 'code': apit.iloc[0].pCode}
        hPitcher = {'name': hpit.iloc[0]['name'], 'code': hpit.iloc[0].pCode}
        away_field['투수'] = aPitcher
        home_field['투수'] = hPitcher
        
        self.away_order = away_order
        self.away_field = away_field
        self.home_order = home_order
        self.home_field = home_field
    
    def 

SyntaxError: invalid syntax (<ipython-input-15-e6079392978f>, line 72)

In [16]:
def parseBatterResult(text):
    return text
def parseRunnerResult(text):
    return text

def parseTextWithType(self, textdf):
    textType = textdf.type
    text = textdf.text
    if textType == 0:
        # 이닝
        inn = int(text.split('회')[0])
        topbot = text.split('회')[1].split(' ')[0]
        topbot = 't' if topbot == '초' else 'b'
        print(inn, topbot)
    elif textType == 1:
        # 투구
        pID = textdf.pitchId
        print(text, pID)
    elif textType == 2:
        # 교체
        # shift, outPlayer, inPlayer
        if np.isnan(textdf.shiftPlayer):
            outPlayer = textdf.outPlayer
            inPlayer = textdf.inPlayer
            # 포지션+이름으로 away / home field, order에서 선수 code 찾기(이름만으로 찾으면 동명이인)
            print(text, outPlayer, inPlayer)
        else:
            shiftPlayer = text.split(' ')[1]
            beforePos = text.split(' ')[0]
            afterPos = text.split(' ')[3].split('(')[0]
            # 포지션+이름으로 away / home field, order에서 선수 code 찾기(이름만으로 찾으면 동명이인)
            # 해당 선수 교체
            print(text, shiftPlayer, beforePos, afterPos)
        

`type`
- `0` : x회초(말) xx 공격
- `1` : 투구 (x구 ...)
- `2` : 교체
- `7` : 시스템 메시지 (비디오 판독, ...)
- `8` : x번타자 xxx
- `13` : 타자주자 타석 결과(득점 없는 타석)
- `14` : 주자 주루/아웃(득점 없는 타석)
- `23` : 타자주자 타석 결과(득점 있는 타석)
- `24` : 주자 주루/아웃(득점 있는 타석)
- `44` : 파울 에러 (pass)
- `99` : 경기 종료 메시지

`pitchResult`
- `T` : s __T__ rike 스트라이크
- `F` : __F__oul 파울
- `S` : __S__wing 헛스윙
- `B` : __B__all 볼
- `H` : __H__it 타격

- `pitchNum` : 문자중계 텍스트(`textOptionList`)에 포함되는 투구수, 정합성 확인 안됨, 신뢰하지 말 것
- `ballcount` : pitch data(`ptsOptionList`)에 포함되는 투구수, `ptsPitchId` 오류 있을 때는 1개씩 밀리는 문제, 신뢰하지 말 것
- `inn` : 문자중계와 pitch data에 모두 포함, 지금은 Pitch data에 있는걸 가져오는 중. 신뢰하지 말 것